In [1]:
#Import de librerías necesarias
import pyreadr
import matplotlib.pyplot as plt
from matplotlib.widgets import Cursor
import numpy as np
import pandas as pd
from datetime import datetime, timedelta, date
from sklearn import *
from sklearn.model_selection import *
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import random
import time
import pandas as pd
from scipy.stats import chi2
from scipy.stats import chi2_contingency
import scipy.stats as stats
from sklearn.metrics import *
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from scipy.stats import mannwhitneyu
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from collections import defaultdict
from statsmodels.stats.outliers_influence import variance_inflation_factor 
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.utils.class_weight import compute_class_weight
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from patsy import cr
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils import resample
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb
import cupy as cp
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import shap

In [2]:
#Importar los datos de R a Python
datos = pyreadr.read_r('area_desbalance_ionico.RData')

#Para comprobar que se importan correctamente los datos
#print(datos.keys())

#Este archivo tiene tres variables

#datos_anon_D: datos extraídos de los ECGS - edad, sexo, fecha, hora y 
#nombre del fichero. Hay un código (código) que identifica a cada paciente
datos_anon_DF = datos['datos_anon_DF']

#lead_anon_DF
#variables electrocardiográficas correspondiente a los ECGs de la variable anterior
leads_anon_DF = datos['leads_anon_DF']

#pacs_con_ECGs_DF
#variables de los datos del potasio, fecha de la analítica, edad del paciente, sexo
#nivel de K y código del paciente
pacs_con_ECGs_DF = datos['pacs_con_ECGs_DF']

# Normalidad - Hiperpotasemia - Hipopotasemia

In [3]:
#Primer paso: coger solo los valores de K normales y de hiperpotasemia
k_val = pacs_con_ECGs_DF.copy()

limites = [float('-inf'), 3.5, 5.2, float('inf')]
etiquetas = [1, 0, 2]

In [4]:
k_valores = k_val.copy()

k_valores['categoria'] = pd.cut(k_valores['K'], bins=limites, labels=etiquetas, right=False)

#Se reemplazan los sexos por enteros
#1: Hombre
#0: Mujer
k_valores['Sexo'] = k_valores['Sexo'].replace({'M': 1, 'F': 0})

# Eliminación de variables con datos vacíos

In [5]:
#En leads_anon_DF, se van a eliminar aquellas columnas en las que falte más del 1% de los datos

#El como máximo (el 100%) de datos que podría tener cada característica, sería igual al número total de filas que
#hay registradas
#Se calcula el 10% y se redondea sin decimales
diez_pct_datos = round(len(leads_anon_DF) * 0.01, 0)

#Ahora, se va a coger un subconjunto del dataframe en el que se eliminan las
#características cuyo número de datos sea menor al 10%

#Se cuenta el número de NaN o celdas sin datos en cada columna
celdas_sin_datos = leads_anon_DF.isnull().sum()

#Se cogen las características cuyo número de datos vacíos es mayor o igual al 10%
caracteristicas_eliminar = celdas_sin_datos[celdas_sin_datos >= diez_pct_datos].index

#Se crear el subDataFrame eliminando las columnas correspondientes
leads_anon_DF_limpio = leads_anon_DF.drop(columns = caracteristicas_eliminar)

In [6]:
#Ahora hay que asociar a cada análisis de sangre, su ECG más cercano
def ECG_mas_reciente(fecha, codigo, dias=5):
     
    archivos_ECGs = datos_anon_DF.loc[(datos_anon_DF['codigo'] == codigo) & (datos_anon_DF['date'] >= (fecha - timedelta(days=dias))) & (datos_anon_DF['date'] <= (fecha + timedelta(days=dias)))]    
    
    fechas_archivos = archivos_ECGs['date']
    horas_archivos = archivos_ECGs['time']
        
    diferencia_temporal = [abs(fecha - fecha_archivo) for fecha_archivo in fechas_archivos]
    
    ecg = (datos_anon_DF.index[(datos_anon_DF['codigo'] == codigo) & (datos_anon_DF['date'] >= (fecha - timedelta(days=dias))) & (datos_anon_DF['date'] <= (fecha + timedelta(days=dias)))]).tolist()
    
    if(len(diferencia_temporal) > 0):
    
        combinado = list(zip(diferencia_temporal, ecg))
        combinado = sorted(combinado, key=lambda x: x[0])

        diferencia_temporal, ecg = zip(*combinado)
        
    return ecg, diferencia_temporal

In [7]:
#Ahora, se añade la columna "Categoria" y "Sexo" a leads_anon_DF_limpio
#Se crean esas columnas en el nuevo dataframe con valores a Nan
leads_anon_DF_limpio['Sexo'] = float('NaN')
leads_anon_DF_limpio['categoria'] = float('NaN')
leads_anon_DF_limpio['K'] = float('NaN')
leads_anon_DF_limpio['Edad'] = float('NaN')

ecgs_utilizadas = {} 

for i in range(len(k_valores)):
    fecha_hora = (k_valores['Fecha'].iloc[i]).to_pydatetime()
    fecha = fecha_hora.date()
    
    indice, dist_temporal = ECG_mas_reciente(fecha, k_valores['codigo'].iloc[i])
  
    for j in range(len(indice)):
        #Si el ECGs no tiene una analítica asignada, se le pone
        if indice[j] not in ecgs_utilizadas:
            #Se actualizan los valores de las columnas "Sexo" y "Categoría"
            #de la fila correspondiente
            leads_anon_DF_limpio.at[indice[j], 'Sexo'] = k_valores['Sexo'].iloc[i]
            leads_anon_DF_limpio.at[indice[j], 'Edad'] = k_valores['Edad'].iloc[i]
            leads_anon_DF_limpio.at[indice[j], 'categoria'] = k_valores['categoria'].iloc[i]
            leads_anon_DF_limpio.at[indice[j], 'K'] = k_valores['K'].iloc[i]

            ecgs_utilizadas[indice[j]] = [k_valores['categoria'].iloc[i], dist_temporal[j].days]

        #Si el ECG ya tiene una analítica asociada, si la nueva analítica tiene un valor
        #de categoría diferente y se asocia con una anormalidad, se cambia
        else:
            if((dist_temporal[j].days < ecgs_utilizadas[indice[j]][1])):
                leads_anon_DF_limpio.at[indice[j], 'categoria'] = k_valores['categoria'].iloc[i]
                leads_anon_DF_limpio.at[indice[j], 'K'] = k_valores['K'].iloc[i]
                ecgs_utilizadas[indice[j]][1] = dist_temporal[j].days
                
            if((dist_temporal[j].days == ecgs_utilizadas[indice[j]][1]) & (k_valores['categoria'].iloc[i] > ecgs_utilizadas[indice[j]][0])):
                leads_anon_DF_limpio.at[indice[j], 'categoria'] = k_valores['categoria'].iloc[i]
                leads_anon_DF_limpio.at[indice[j], 'K'] = k_valores['K'].iloc[i]
                ecgs_utilizadas[indice[j]][1] = dist_temporal[j].days                
            
#Se eliminan las filas que tengan la variable objetivo "categoria" a NaN
leads_anon_DF_limpio = leads_anon_DF_limpio.dropna(subset=['categoria'])

In [8]:
pparea = leads_anon_DF_limpio.filter(regex='_pparea', axis=1)
tparea = leads_anon_DF_limpio.filter(regex='_tparea', axis=1)

ppamp = leads_anon_DF_limpio.filter(regex='_ppamp', axis=1)
rpamp = leads_anon_DF_limpio.filter(regex='_rpamp', axis=1)
spamp = leads_anon_DF_limpio.filter(regex='_spamp', axis=1)
tpamp = leads_anon_DF_limpio.filter(regex='_tpamp', axis=1)

rpdur = leads_anon_DF_limpio.filter(regex='_rpdur', axis=1)
spdur = leads_anon_DF_limpio.filter(regex='_spdur', axis=1)
tpdur = leads_anon_DF_limpio.filter(regex='_tpdur', axis=1)

leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=pparea.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=tparea.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=ppamp.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=rpamp.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=spamp.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=tpamp.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=rpdur.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=spdur.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=tpdur.columns)

In [9]:
parea = leads_anon_DF_limpio.filter(regex='_parea', axis=1)
pppparea = leads_anon_DF_limpio.filter(regex='_pppparea', axis=1)
qdur = leads_anon_DF_limpio.filter(regex='_qdur', axis=1)
sdur = leads_anon_DF_limpio.filter(regex='_sdur', axis=1)
tarea = leads_anon_DF_limpio.filter(regex='_tarea', axis=1)
tptparea = leads_anon_DF_limpio.filter(regex='(_tptparea$)', axis=1)
tptpdur = leads_anon_DF_limpio.filter(regex='(_tptpdur$)', axis=1)
st = leads_anon_DF_limpio.filter(regex='(_stend$|_st80$|_ston$)', axis=1)
stslope = leads_anon_DF_limpio.filter(regex='(_stslope)', axis=1)
stdur = leads_anon_DF_limpio.filter(regex='(_stdur$)', axis=1).drop(columns=['V2_stdur'])

leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=parea.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=pppparea.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=qdur.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=sdur.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=tarea.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=tptparea.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=tptpdur.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=st.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=stslope.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=stdur.columns)

# CARGA DE DATOS COMPLETADA

### Hipopotasemia

In [10]:
x = leads_anon_DF_limpio[leads_anon_DF_limpio['categoria'] != 2]
y = x['categoria'].astype(int)
y = y.reset_index(drop=True)

estratificacion_log = x[['categoria', 'Sexo']]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1, stratify=estratificacion_log)

x_train = x_train.drop(['K', 'categoria'], axis = 1)
x_test = x_test.drop(['K', 'categoria'], axis = 1)
x_train = x_train.where(pd.notna(x_train), None)
x_test = x_test.where(pd.notna(x_test), None)

imputer = IterativeImputer(max_iter=5, sample_posterior=True, initial_strategy='mean', skip_complete=True)

x_train = pd.DataFrame(imputer.fit_transform(x_train), columns=x_train.columns)
x_test = pd.DataFrame(imputer.transform(x_test), columns=x_train.columns)

esc = StandardScaler()
x_train = pd.DataFrame(esc.fit_transform(x_train), columns=x_train.columns)
x_test = pd.DataFrame(esc.transform(x_test), columns=x_test.columns)

x = x.drop(['K', 'categoria'], axis = 1)
x = x.where(pd.notna(x), None)

imputer = IterativeImputer(max_iter=5, sample_posterior=True, initial_strategy='mean', skip_complete=True)

x = pd.DataFrame(imputer.fit_transform(x), columns=x.columns)

esc = StandardScaler()
x = pd.DataFrame(esc.fit_transform(x), columns=x.columns)

### Hiperpotasemia

In [36]:
x = leads_anon_DF_limpio[leads_anon_DF_limpio['categoria'] != 1]
y = x['categoria'].astype(int)

y = y.apply(lambda x: 1 if x == 2 else 0)
y = y.reset_index(drop=True)

estratificacion_log = x[['categoria', 'Sexo']]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1, stratify=estratificacion_log)

x_train = x_train.drop(['K', 'categoria'], axis = 1)
x_test = x_test.drop(['K', 'categoria'], axis = 1)
x_train = x_train.where(pd.notna(x_train), None)
x_test = x_test.where(pd.notna(x_test), None)

imputer = IterativeImputer(max_iter=5, sample_posterior=True, initial_strategy='mean', skip_complete=True)

x_train = pd.DataFrame(imputer.fit_transform(x_train), columns=x_train.columns)
x_test = pd.DataFrame(imputer.transform(x_test), columns=x_train.columns)

esc = StandardScaler()
x_train = pd.DataFrame(esc.fit_transform(x_train), columns=x_train.columns)
x_test = pd.DataFrame(esc.transform(x_test), columns=x_test.columns)

x = x.drop(['K', 'categoria'], axis = 1)
x = x.where(pd.notna(x), None)

imputer = IterativeImputer(max_iter=5, sample_posterior=True, initial_strategy='mean', skip_complete=True)

x = pd.DataFrame(imputer.fit_transform(x), columns=x.columns)

esc = StandardScaler()
x = pd.DataFrame(esc.fit_transform(x), columns=x.columns)

### Regresión

In [44]:
x = leads_anon_DF_limpio
x = x.sample(frac=1, random_state=1)
y = x['K']

y = y.reset_index(drop=True)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

x_train = x_train.drop(['K', 'categoria'], axis = 1)
x_test = x_test.drop(['K', 'categoria'], axis = 1)
x_train = x_train.where(pd.notna(x_train), None)
x_test = x_test.where(pd.notna(x_test), None)

imputer = IterativeImputer(max_iter=5, sample_posterior=True, initial_strategy='mean', skip_complete=True)

x_train = pd.DataFrame(imputer.fit_transform(x_train), columns=x_train.columns)
x_test = pd.DataFrame(imputer.transform(x_test), columns=x_train.columns)

esc = StandardScaler()
x_train = pd.DataFrame(esc.fit_transform(x_train), columns=x_train.columns)
x_test = pd.DataFrame(esc.transform(x_test), columns=x_test.columns)

x = x.drop(['K', 'categoria'], axis = 1)
x = x.where(pd.notna(x), None)

imputer = IterativeImputer(max_iter=5, sample_posterior=True, initial_strategy='mean', skip_complete=True)

x = pd.DataFrame(imputer.fit_transform(x), columns=x.columns)

esc = StandardScaler()
x = pd.DataFrame(esc.fit_transform(x), columns=x.columns)

---

# Clasificación

## Hipopotasemia

### Regresión logística

In [32]:
cars_aic = ['aVL_pamp', 'II_ramp', 'V5_rdur', 'aVL_vat', 'V3_vat', 'aVL_qrsppk', 'V1_qrsppk', 'III_qrsdur', 'V1_qrsdur', 'V3_stmid', 'V6_stmid', 'V2_stdur', 'aVR_tamp', 'V1_tamp', 'V5_tamp', 'I_tdur', 'V6_tdur']
cars_bor = ['aVL_pamp', 'V4_ramp', 'I_stmid', 'aVR_stmid', 'V2_stmid', 'V3_stmid', 'V2_stdur', 'aVR_tamp', 'V2_tamp', 'V3_tamp']

caracteristicas = [cars_aic, cars_bor]
models = ['AIC', 'Boruta']

for i in range(len(caracteristicas)):
    
    x2 = x[caracteristicas[i]]
    x2 = sm.add_constant(x2)
    
    aucs = []
    exs = []
    precs = []
    esps = []
    sens = []

    for j in range(100):
        
        kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=j)
        
        for fold, (train, test) in enumerate(kf.split(x2, y)):
            
            x_t, x_ts = x2.values[train], x2.values[test]
            y_t, y_ts = y[train], y[test]
            
            y_t = y_t.reset_index(drop=True)
            y_ts = y_ts.reset_index(drop=True)

            class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y_t), y=y_t)
            pesos = y_t.apply(lambda z: class_weights[z])
        
            modelo = sm.GLM(y_t, x_t, family=sm.families.Binomial(), freq_weights=pesos)
            modelo = modelo.fit()
            
            prob = modelo.predict(x_ts)
            pred = (prob >= 0.5).astype(int)
            tn, fp, fn, tp = confusion_matrix(y_ts, pred).ravel()
            auc = roc_auc_score(y_ts, prob)
            prec = precision_score(y_ts, pred)
            ex = accuracy_score(y_ts, pred)
            sen = recall_score(y_ts, pred)
            esp = tn / (tn + fp)
            
            aucs.append(auc)
            exs.append(ex)
            precs.append(prec)
            sens.append(sen)
            esps.append(esp)        
    
    print(f' ')
    print(f'RESULTADOS DEL CONJUNTO SELECCIONADO POR {models[i]}')
    print(f'---------------------------------------------')
    print(f'Acierto: {np.mean(exs)}')
    print(f'Precision: {np.mean(precs)}')
    print(f'Sensibilidad: {np.mean(sens)}')
    print(f'Especificidad: {np.mean(esps)}')
    print(f'AUC: {np.mean(aucs)}')
    
    x_train_2 = x_train[caracteristicas[i]]
    x_train_2 = sm.add_constant(x_train_2)
    
    class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y_train), y=y_train)
    pesos = y_train.apply(lambda z: class_weights[z])

    y_train = y_train.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)

    modelo = sm.GLM(y_train, x_train_2, family=sm.families.Binomial(), freq_weights=pesos)
    modelo = modelo.fit()
    print(modelo.summary())

 
RESULTADOS DEL CONJUNTO SELECCIONADO POR AIC
---------------------------------------------
Acierto: 0.7093228998350852
Precision: 0.2225424747443344
Sensibilidad: 0.6665120910384069
Especificidad: 0.7145580632959273
AUC: 0.7574959396415707
                 Generalized Linear Model Regression Results                  
Dep. Variable:              categoria   No. Observations:                 1386
Model:                            GLM   Df Residuals:                     1368
Model Family:                Binomial   Df Model:                           17
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -780.50
Date:                Mon, 01 Jul 2024   Deviance:                       1561.0
Time:                        11:49:35   Pearson chi2:                 1.58e+03
No. Iterations:                     5   Pseudo R-squ. (CS):             0.2290
Covariance Type:            nonrobust          

### Árbol de decisión

In [27]:
cars_aic = ['aVL_pamp', 'II_ramp', 'V5_rdur', 'aVL_vat', 'V3_vat', 'aVL_qrsppk', 'V1_qrsppk', 'III_qrsdur', 'V1_qrsdur', 'V3_stmid', 'V6_stmid', 'V2_stdur', 'aVR_tamp', 'V1_tamp', 'V5_tamp', 'I_tdur', 'V6_tdur']
cars_bor = ['aVL_pamp', 'V4_ramp', 'I_stmid', 'aVR_stmid', 'V2_stmid', 'V3_stmid', 'V2_stdur', 'aVR_tamp', 'V2_tamp', 'V3_tamp']

caracteristicas = [cars_aic, cars_bor]
models = ['AIC', 'Boruta']

params_aic = {'max_depth': 10, 'max_features': 10, 'max_leaf_nodes': 10}
params_bor = {'max_depth': None, 'max_features': 5, 'max_leaf_nodes': 10}
params = [params_aic, params_bor]

modelos = []
val_test = []

for i in range(len(caracteristicas)):
    
    x2 = x[caracteristicas[i]]
    
    aucs = []
    exs = []
    precs = []
    esps = []
    sens = []

    for j in range(100):
        
        kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=j)
        
        for fold, (train, test) in enumerate(kf.split(x2, y)):
            
            x_t, x_ts = x2.values[train], x2.values[test]
            y_t, y_ts = y[train], y[test]
            
            y_t = y_t.reset_index(drop=True)
            y_ts = y_ts.reset_index(drop=True)

            modelo = DecisionTreeClassifier(class_weight='balanced',  max_leaf_nodes=params[i]['max_leaf_nodes'], 
                                   max_features=params[i]['max_features'], max_depth=params[i]['max_depth'])
            modelo.fit(x_t, y_t)
            
            prob = modelo.predict_proba(x_ts)
            pred = modelo.predict(x_ts)
            tn, fp, fn, tp = confusion_matrix(y_ts, pred).ravel()
            auc = roc_auc_score(y_ts, prob[:,1])
            prec = precision_score(y_ts, pred)
            ex = accuracy_score(y_ts, pred)
            sen = recall_score(y_ts, pred)
            esp = tn / (tn + fp)
            
            aucs.append(auc)
            exs.append(ex)
            precs.append(prec)
            sens.append(sen)
            esps.append(esp)        
    
    print(f' ')
    print(f'RESULTADOS DEL CONJUNTO SELECCIONADO POR {models[i]}')
    print(f'---------------------------------------------')
    print(f'Acierto: {np.mean(exs)}')
    print(f'Precision: {np.mean(precs)}')
    print(f'Sensibilidad: {np.mean(sens)}')
    print(f'Especificidad: {np.mean(esps)}')
    print(f'AUC: {np.mean(aucs)}')
    
    x_train_2 = x_train[caracteristicas[i]]
    
    modelo = DecisionTreeClassifier(class_weight='balanced',  max_leaf_nodes=params[i]['max_leaf_nodes'], 
                                   max_features=params[i]['max_features'], max_depth=params[i]['max_depth'])
    modelo.fit(x_train_2, y_train)
    
    modelos.append(modelo)
    val_test.append(x_test[caracteristicas[i]])
    
    cars_x = x_train_2.columns
    importancias = modelo.feature_importances_

    car_imp = sorted(zip(cars_x, importancias), key=lambda x: x[1], reverse=True)

    print(f' ')
    print(f'IMPORTANCIAS')
    print(f'--------------------------------')
    for car, importancia in car_imp:
        print(f'{car}: {importancia}')
    

 
RESULTADOS DEL CONJUNTO SELECCIONADO POR AIC
---------------------------------------------
Acierto: 0.6009091802568673
Precision: 0.15715957290564103
Sensibilidad: 0.5935988620199147
Especificidad: 0.601783087462699
AUC: 0.623970394628481
 
IMPORTANCIAS
--------------------------------
V3_stmid: 0.2954587002497594
V6_stmid: 0.19968993360895565
aVL_pamp: 0.17485854196888168
V3_vat: 0.10261277874816432
aVL_qrsppk: 0.09295713733213991
III_qrsdur: 0.06761134017806782
V1_tamp: 0.06681156791403119
II_ramp: 0.0
V5_rdur: 0.0
aVL_vat: 0.0
V1_qrsppk: 0.0
V1_qrsdur: 0.0
V2_stdur: 0.0
aVR_tamp: 0.0
V5_tamp: 0.0
I_tdur: 0.0
V6_tdur: 0.0
 
RESULTADOS DEL CONJUNTO SELECCIONADO POR Boruta
---------------------------------------------
Acierto: 0.6103266312405258
Precision: 0.16405375918501391
Sensibilidad: 0.6068406827880513
Especificidad: 0.610753288782415
AUC: 0.6425494303224247
 
IMPORTANCIAS
--------------------------------
aVR_stmid: 0.2704459896094087
aVR_tamp: 0.22275203312170938
aVL_pamp: 0.2

### Random Forest

In [16]:
cars_aic = ['aVL_pamp', 'II_ramp', 'V5_rdur', 'aVL_vat', 'V3_vat', 'aVL_qrsppk', 'V1_qrsppk', 'III_qrsdur', 'V1_qrsdur', 'V3_stmid', 'V6_stmid', 'V2_stdur', 'aVR_tamp', 'V1_tamp', 'V5_tamp', 'I_tdur', 'V6_tdur']
cars_bor = ['aVL_pamp', 'V4_ramp', 'I_stmid', 'aVR_stmid', 'V2_stmid', 'V3_stmid', 'V2_stdur', 'aVR_tamp', 'V2_tamp', 'V3_tamp']

caracteristicas = [cars_aic, cars_bor]
models = ['AIC', 'Boruta']

params_aic = {'n_estimators': 150, 'max_leaf_nodes': None, 'max_features': None, 'max_depth': 25}
params_bor = {'n_estimators': 150, 'max_leaf_nodes': None, 'max_features': None, 'max_depth': None}
params = [params_aic, params_bor]

modelos = []
val_test = []

for i in range(len(caracteristicas)):
    
    x2 = x[caracteristicas[i]]
    
    aucs = []
    exs = []
    precs = []
    esps = []
    sens = []

    for j in range(3):
        
        kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=j)
        
        for fold, (train, test) in enumerate(kf.split(x2, y)):
            
            x_t, x_ts = x2.values[train], x2.values[test]
            y_t, y_ts = y[train], y[test]
            
            y_t = y_t.reset_index(drop=True)
            y_ts = y_ts.reset_index(drop=True)

            modelo = RandomForestClassifier(n_jobs=-1, oob_score=roc_auc_score, class_weight='balanced',
                                   n_estimators=params[i]['n_estimators'],  
                                   max_leaf_nodes=params[i]['max_leaf_nodes'], max_features=params[i]['max_features'], 
                                            max_depth=params[i]['max_depth'])
            modelo.fit(x_t, y_t)
            
            prob = modelo.predict_proba(x_ts)
            pred = modelo.predict(x_ts)
                      
            tn, fp, fn, tp = confusion_matrix(y_ts, pred).ravel()
            auc = roc_auc_score(y_ts, prob[:,1])
            prec = precision_score(y_ts, pred, zero_division=0)
            ex = accuracy_score(y_ts, pred)
            sen = recall_score(y_ts, pred)
            esp = tn / (tn + fp)
            
            aucs.append(auc)
            exs.append(ex)
            precs.append(prec)
            sens.append(sen)
            esps.append(esp)        
    
    print(f' ')
    print(f'RESULTADOS DEL CONJUNTO SELECCIONADO POR {models[i]}')
    print(f'---------------------------------------------')
    print(f'Acierto: {np.mean(exs)}')
    print(f'Precision: {np.mean(precs)}')
    print(f'Sensibilidad: {np.mean(sens)}')
    print(f'Especificidad: {np.mean(esps)}')
    print(f'AUC: {np.mean(aucs)}')
    
    x_train_2 = x_train[caracteristicas[i]]
    
    modelo = RandomForestClassifier(n_jobs=-1, oob_score=roc_auc_score, class_weight='balanced',
                                   n_estimators=params[i]['n_estimators'],  
                                   max_leaf_nodes=params[i]['max_leaf_nodes'], max_features=params[i]['max_features'], 
                                            max_depth=params[i]['max_depth'])
    modelo.fit(x_train_2, y_train)
    
    modelos.append(modelo)
    val_test.append(x_test[caracteristicas[i]])
    
    cars_x = x_train_2.columns
    importancias = modelo.feature_importances_

    car_imp = sorted(zip(cars_x, importancias), key=lambda x: x[1], reverse=True)

    print(f' ')
    print(f'IMPORTANCIAS')
    print(f'--------------------------------')
    for car, importancia in car_imp:
        print(f'{car}: {importancia}')
    

 
RESULTADOS DEL CONJUNTO SELECCIONADO POR AIC
---------------------------------------------
Acierto: 0.8899790663712639
Precision: 0.3722222222222222
Sensibilidad: 0.022901849217638694
Especificidad: 0.9961144033959569
AUC: 0.7582083939582662
 
IMPORTANCIAS
--------------------------------
V3_stmid: 0.11905364834718758
V1_qrsppk: 0.07444709541374193
aVR_tamp: 0.07146059772742663
V2_stdur: 0.06707106832470945
aVL_pamp: 0.0650076449283793
aVL_qrsppk: 0.06322220378382333
V1_tamp: 0.059919903597110365
I_tdur: 0.05968616765662411
II_ramp: 0.059629036278684315
V6_stmid: 0.053874389952131146
V5_rdur: 0.05099539804116737
V3_vat: 0.04817279638551269
III_qrsdur: 0.04725187757053388
V6_tdur: 0.04554085607591673
V5_tamp: 0.04415719896930397
aVL_vat: 0.04198458071855958
V1_qrsdur: 0.028525536229187617
 
RESULTADOS DEL CONJUNTO SELECCIONADO POR Boruta
---------------------------------------------
Acierto: 0.8922895392935872
Precision: 0.6999999999999998
Sensibilidad: 0.0458511142721669
Especificida

### XGBoost

In [15]:
cars_aic = ['aVL_pamp', 'II_ramp', 'V5_rdur', 'aVL_vat', 'V3_vat', 'aVL_qrsppk', 'V1_qrsppk', 'III_qrsdur', 'V1_qrsdur', 'V3_stmid', 'V6_stmid', 'V2_stdur', 'aVR_tamp', 'V1_tamp', 'V5_tamp', 'I_tdur', 'V6_tdur']
cars_bor = ['aVL_pamp', 'V4_ramp', 'I_stmid', 'aVR_stmid', 'V2_stmid', 'V3_stmid', 'V2_stdur', 'aVR_tamp', 'V2_tamp', 'V3_tamp']

caracteristicas = [cars_aic, cars_bor]
models = ['AIC', 'Boruta']

params_aic = {'n_estimators': 100, 'max_leaves': 50, 'max_depth': 50, 'learning_rate': 0.5}
params_bor = {'n_estimators': 250, 'max_leaves': 25, 'max_depth': 25, 'learning_rate': 0.1}
params = [params_aic, params_bor]

modelos = []
val_test = []

for i in range(len(caracteristicas)):
    
    x2 = x[caracteristicas[i]]
    
    aucs = []
    exs = []
    precs = []
    esps = []
    sens = []

    for j in range(100):
        
        kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=j)
        
        for fold, (train, test) in enumerate(kf.split(x2, y)):
            
            x_t, x_ts = x2.values[train], x2.values[test]
            y_t, y_ts = y[train], y[test]
            
            y_t = y_t.reset_index(drop=True)
            y_ts = y_ts.reset_index(drop=True)
            
            pesos = (len(y_t[y_t == 0]) / len(y_t[y_t==1]))

            modelo = xgb.XGBClassifier(device="cuda", n_jobs=-1, eval_metric=roc_auc_score, scale_pos_weight=pesos, 
                              n_estimators=params[i]['n_estimators'], max_leaves=params[i]['max_leaves'], 
                                       max_depth=params[i]['max_depth'], learning_rate=params[i]['learning_rate'])
            modelo.fit(cp.array(x_t), y_t)
            
            prob = modelo.predict_proba(cp.array(x_ts))
            pred = modelo.predict(cp.array(x_ts))
                      
            tn, fp, fn, tp = confusion_matrix(y_ts, pred).ravel()
            auc = roc_auc_score(y_ts, prob[:,1])
            prec = precision_score(y_ts, pred, zero_division=0)
            ex = accuracy_score(y_ts, pred)
            sen = recall_score(y_ts, pred)
            esp = tn / (tn + fp)
            
            aucs.append(auc)
            exs.append(ex)
            precs.append(prec)
            sens.append(sen)
            esps.append(esp)        
    
    print(f' ')
    print(f'RESULTADOS DEL CONJUNTO SELECCIONADO POR {models[i]}')
    print(f'---------------------------------------------')
    print(f'Acierto: {np.mean(exs)}')
    print(f'Precision: {np.mean(precs)}')
    print(f'Sensibilidad: {np.mean(sens)}')
    print(f'Especificidad: {np.mean(esps)}')
    print(f'AUC: {np.mean(aucs)}')
    
    x_train_2 = x_train[caracteristicas[i]]
    
    modelo = xgb.XGBClassifier(device="cuda", n_jobs=-1, eval_metric=roc_auc_score, scale_pos_weight=pesos, 
                              n_estimators=params[i]['n_estimators'], max_leaves=params[i]['max_leaves'], 
                                       max_depth=params[i]['max_depth'], learning_rate=params[i]['learning_rate'])
    modelo.fit(cp.array(x_train_2), y_train)
    
    modelos.append(modelo)
    val_test.append(x_test[caracteristicas[i]])
    
    cars_x = x_train_2.columns
    importancias = modelo.feature_importances_

    car_imp = sorted(zip(cars_x, importancias), key=lambda x: x[1], reverse=True)

    print(f' ')
    print(f'IMPORTANCIAS')
    print(f'--------------------------------')
    for car, importancia in car_imp:
        print(f'{car}: {importancia}')
    

 
RESULTADOS DEL CONJUNTO SELECCIONADO POR AIC
---------------------------------------------
Acierto: 0.8736284604574831
Precision: 0.3271604938271605
Sensibilidad: 0.1587301587301587
Especificidad: 0.9611411229899388
AUC: 0.7355437986543725
 
IMPORTANCIAS
--------------------------------
V3_stmid: 0.0877031460404396
III_qrsdur: 0.08704829961061478
aVR_tamp: 0.0792413279414177
aVL_qrsppk: 0.07015573978424072
V5_rdur: 0.06747163832187653
V2_stdur: 0.06610053032636642
V1_tamp: 0.06371141970157623
V1_qrsppk: 0.05423543602228165
aVL_pamp: 0.052718572318553925
V6_stmid: 0.05176522582769394
V5_tamp: 0.050082314759492874
aVL_vat: 0.04778708890080452
V6_tdur: 0.04754563048481941
V1_qrsdur: 0.04696018248796463
II_ramp: 0.04671202972531319
I_tdur: 0.042891666293144226
V3_vat: 0.037869714200496674
 
RESULTADOS DEL CONJUNTO SELECCIONADO POR Boruta
---------------------------------------------
Acierto: 0.8672827475367758
Precision: 0.28763309990138436
Sensibilidad: 0.13756613756613756
Especificidad

---

## Hiperpotasemia

### Regresión logística

In [40]:
cars_aic = ['V5_qamp', 'I_rdur', 'V5_samp', 'V5_vat', 'V1_qrsdur', 'aVL_qrsarea', 'V1_qrsarea', 'V3_qrsarea', 'V3_stmid', 'V2_tamp', 'V6_qtint']
cars_bor = ['I_ramp', 'V5_samp', 'I_qrsppk', 'II_qrsppk', 'aVR_qrsppk', 'V1_qrsppk',
'V3_qrsppk', 'V4_qrsppk', 'V5_qrsppk', 'V1_qrsdur', 'V5_qrsarea', 'V4_stmid',
'V2_tamp', 'V3_tamp', 'aVL_tdur', 'III_qtint', 'V6_qtint', 'Edad'
]

caracteristicas = [cars_aic, cars_bor]
models = ['AIC', 'Boruta']

for i in range(len(caracteristicas)):
    
    x2 = x[caracteristicas[i]]
    x2 = sm.add_constant(x2)
    
    aucs = []
    exs = []
    precs = []
    esps = []
    sens = []

    for j in range(100):
        
        kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=j)
        
        for fold, (train, test) in enumerate(kf.split(x2, y)):
            
            x_t, x_ts = x2.values[train], x2.values[test]
            y_t, y_ts = y[train], y[test]
            
            y_t = y_t.reset_index(drop=True)
            y_ts = y_ts.reset_index(drop=True)

            class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y_t), y=y_t)
            pesos = y_t.apply(lambda z: class_weights[z])
        
            modelo = sm.GLM(y_t, x_t, family=sm.families.Binomial(), freq_weights=pesos)
            modelo = modelo.fit()
            
            prob = modelo.predict(x_ts)
            pred = (prob >= 0.5).astype(int)
            tn, fp, fn, tp = confusion_matrix(y_ts, pred).ravel()
            auc = roc_auc_score(y_ts, prob)
            prec = precision_score(y_ts, pred)
            ex = accuracy_score(y_ts, pred)
            sen = recall_score(y_ts, pred)
            esp = tn / (tn + fp)
            
            aucs.append(auc)
            exs.append(ex)
            precs.append(prec)
            sens.append(sen)
            esps.append(esp)        
    
    print(f' ')
    print(f'RESULTADOS DEL CONJUNTO SELECCIONADO POR {models[i]}')
    print(f'---------------------------------------------')
    print(f'Acierto: {np.mean(exs)}')
    print(f'Precision: {np.mean(precs)}')
    print(f'Sensibilidad: {np.mean(sens)}')
    print(f'Especificidad: {np.mean(esps)}')
    print(f'AUC: {np.mean(aucs)}')
    
    x_train_2 = x_train[caracteristicas[i]]
    x_train_2 = sm.add_constant(x_train_2)
    
    class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y_train), y=y_train)
    pesos = y_train.apply(lambda z: class_weights[z])

    y_train = y_train.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)

    modelo = sm.GLM(y_train, x_train_2, family=sm.families.Binomial(), freq_weights=pesos)
    modelo = modelo.fit()
    print(modelo.summary())

 
RESULTADOS DEL CONJUNTO SELECCIONADO POR AIC
---------------------------------------------
Acierto: 0.610702761842243
Precision: 0.43125357496205674
Sensibilidad: 0.5959333333333334
Especificidad: 0.6178768335224646
AUC: 0.6448993574440662
                 Generalized Linear Model Regression Results                  
Dep. Variable:              categoria   No. Observations:                 1835
Model:                            GLM   Df Residuals:                     1823
Model Family:                Binomial   Df Model:                           11
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1201.2
Date:                Mon, 01 Jul 2024   Deviance:                       2402.3
Time:                        12:01:09   Pearson chi2:                 1.89e+03
No. Iterations:                     4   Pseudo R-squ. (CS):            0.07422
Covariance Type:            nonrobust          

### Árbol de decisión

In [41]:
cars_aic = ['V5_qamp', 'I_rdur', 'V5_samp', 'V5_vat', 'V1_qrsdur', 'aVL_qrsarea', 'V1_qrsarea', 'V3_qrsarea', 'V3_stmid', 'V2_tamp', 'V6_qtint']
cars_bor = ['I_ramp', 'V5_samp', 'I_qrsppk', 'II_qrsppk', 'aVR_qrsppk', 'V1_qrsppk',
'V3_qrsppk', 'V4_qrsppk', 'V5_qrsppk', 'V1_qrsdur', 'V5_qrsarea', 'V4_stmid',
'V2_tamp', 'V3_tamp', 'aVL_tdur', 'III_qtint', 'V6_qtint', 'Edad'
]

caracteristicas = [cars_aic, cars_bor]
models = ['AIC', 'Boruta']

params_aic = {'max_depth': None, 'max_features': 10, 'max_leaf_nodes': 50}
params_bor = {'max_depth': 100, 'max_features': 10, 'max_leaf_nodes': 50}
params = [params_aic, params_bor]

for i in range(len(caracteristicas)):
    
    x2 = x[caracteristicas[i]]
    
    aucs = []
    exs = []
    precs = []
    esps = []
    sens = []

    for j in range(100):
        
        kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=j)
        
        for fold, (train, test) in enumerate(kf.split(x2, y)):
            
            x_t, x_ts = x2.values[train], x2.values[test]
            y_t, y_ts = y[train], y[test]
            
            y_t = y_t.reset_index(drop=True)
            y_ts = y_ts.reset_index(drop=True)

            modelo = DecisionTreeClassifier(class_weight='balanced',  max_leaf_nodes=params[i]['max_leaf_nodes'], 
                                   max_features=params[i]['max_features'], max_depth=params[i]['max_depth'])
            modelo.fit(x_t, y_t)
            
            prob = modelo.predict_proba(x_ts)
            pred = modelo.predict(x_ts)
            tn, fp, fn, tp = confusion_matrix(y_ts, pred).ravel()
            auc = roc_auc_score(y_ts, prob[:,1])
            prec = precision_score(y_ts, pred)
            ex = accuracy_score(y_ts, pred)
            sen = recall_score(y_ts, pred)
            esp = tn / (tn + fp)
            
            aucs.append(auc)
            exs.append(ex)
            precs.append(prec)
            sens.append(sen)
            esps.append(esp)        
    
    print(f' ')
    print(f'RESULTADOS DEL CONJUNTO SELECCIONADO POR {models[i]}')
    print(f'---------------------------------------------')
    print(f'Acierto: {np.mean(exs)}')
    print(f'Precision: {np.mean(precs)}')
    print(f'Sensibilidad: {np.mean(sens)}')
    print(f'Especificidad: {np.mean(esps)}')
    print(f'AUC: {np.mean(aucs)}')
    
    x_train_2 = x_train[caracteristicas[i]]
    
    modelo = DecisionTreeClassifier(class_weight='balanced',  max_leaf_nodes=params[i]['max_leaf_nodes'], 
                                   max_features=params[i]['max_features'], max_depth=params[i]['max_depth'])
    modelo.fit(x_train_2, y_train)
    
    cars_x = x_train_2.columns
    importancias = modelo.feature_importances_

    car_imp = sorted(zip(cars_x, importancias), key=lambda x: x[1], reverse=True)

    print(f' ')
    print(f'IMPORTANCIAS')
    print(f'--------------------------------')
    for car, importancia in car_imp:
        print(f'{car}: {importancia}')
    

 
RESULTADOS DEL CONJUNTO SELECCIONADO POR AIC
---------------------------------------------
Acierto: 0.578886986138463
Precision: 0.40416206030573926
Sensibilidad: 0.5960933333333335
Especificidad: 0.5705293363594335
AUC: 0.6032347827092002
 
IMPORTANCIAS
--------------------------------
V2_tamp: 0.1923431226750317
I_rdur: 0.12468268332570076
V5_vat: 0.1242254858025355
V1_qrsdur: 0.11093907396691642
aVL_qrsarea: 0.0984915379420651
V6_qtint: 0.09027324396841789
V3_qrsarea: 0.07136891235210772
V3_stmid: 0.06740609959502201
V1_qrsarea: 0.06624542346771388
V5_samp: 0.05402441690448896
V5_qamp: 0.0
 
RESULTADOS DEL CONJUNTO SELECCIONADO POR Boruta
---------------------------------------------
Acierto: 0.5911771555783885
Precision: 0.412064261441492
Sensibilidad: 0.5760533333333333
Especificidad: 0.5985203841465977
AUC: 0.6057106233626136
 
IMPORTANCIAS
--------------------------------
V4_qrsppk: 0.13444113541195213
V4_stmid: 0.08298122077148778
V1_qrsppk: 0.07740460926378685
V2_tamp: 0.076

### Random Forest

In [24]:
cars_aic = ['V5_qamp', 'I_rdur', 'V5_samp', 'V5_vat', 'V1_qrsdur', 'aVL_qrsarea', 'V1_qrsarea', 'V3_qrsarea', 'V3_stmid', 'V2_tamp', 'V6_qtint']
cars_bor = ['I_ramp', 'V5_samp', 'I_qrsppk', 'II_qrsppk', 'aVR_qrsppk', 'V1_qrsppk',
'V3_qrsppk', 'V4_qrsppk', 'V5_qrsppk', 'V1_qrsdur', 'V5_qrsarea', 'V4_stmid',
'V2_tamp', 'V3_tamp', 'aVL_tdur', 'III_qtint', 'V6_qtint', 'Edad'
]

caracteristicas = [cars_aic, cars_bor]
models = ['AIC', 'Boruta']

params_aic = {'n_estimators': 200, 'max_leaf_nodes': None, 'max_features': None, 'max_depth': 100}
params_bor = {'n_estimators': 200, 'max_leaf_nodes': None, 'max_features': 5, 'max_depth': None}
params = [params_aic, params_bor]

for i in range(len(caracteristicas)):
    
    x2 = x[caracteristicas[i]]
    
    aucs = []
    exs = []
    precs = []
    esps = []
    sens = []

    for j in range(100):
        
        kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=j)
        
        for fold, (train, test) in enumerate(kf.split(x2, y)):
            
            x_t, x_ts = x2.values[train], x2.values[test]
            y_t, y_ts = y[train], y[test]
            
            y_t = y_t.reset_index(drop=True)
            y_ts = y_ts.reset_index(drop=True)

            modelo = RandomForestClassifier(n_jobs=-1, oob_score=roc_auc_score, class_weight='balanced',
                                   n_estimators=params[i]['n_estimators'],  
                                   max_leaf_nodes=params[i]['max_leaf_nodes'], max_features=params[i]['max_features'], 
                                            max_depth=params[i]['max_depth'])
            modelo.fit(x_t, y_t)
            
            prob = modelo.predict_proba(x_ts)
            pred = modelo.predict(x_ts)
                      
            tn, fp, fn, tp = confusion_matrix(y_ts, pred).ravel()
            auc = roc_auc_score(y_ts, prob[:,1])
            prec = precision_score(y_ts, pred, zero_division=0)
            ex = accuracy_score(y_ts, pred)
            sen = recall_score(y_ts, pred)
            esp = tn / (tn + fp)
            
            aucs.append(auc)
            exs.append(ex)
            precs.append(prec)
            sens.append(sen)
            esps.append(esp)        
    
    print(f' ')
    print(f'RESULTADOS DEL CONJUNTO SELECCIONADO POR {models[i]}')
    print(f'---------------------------------------------')
    print(f'Acierto: {np.mean(exs)}')
    print(f'Precision: {np.mean(precs)}')
    print(f'Sensibilidad: {np.mean(sens)}')
    print(f'Especificidad: {np.mean(esps)}')
    print(f'AUC: {np.mean(aucs)}')
    
    x_train_2 = x_train[caracteristicas[i]]
    
    modelo = RandomForestClassifier(n_jobs=-1, oob_score=roc_auc_score, class_weight='balanced',
                                   n_estimators=params[i]['n_estimators'],  
                                   max_leaf_nodes=params[i]['max_leaf_nodes'], max_features=params[i]['max_features'], 
                                            max_depth=params[i]['max_depth'])
    modelo.fit(x_train_2, y_train)
    
    cars_x = x_train_2.columns
    importancias = modelo.feature_importances_

    car_imp = sorted(zip(cars_x, importancias), key=lambda x: x[1], reverse=True)

    print(f' ')
    print(f'IMPORTANCIAS')
    print(f'--------------------------------')
    for car, importancia in car_imp:
        print(f'{car}: {importancia}')
    

 
RESULTADOS DEL CONJUNTO SELECCIONADO POR AIC
---------------------------------------------
Acierto: 0.7080652262845943
Precision: 0.6323617577601934
Sensibilidad: 0.25938666666666665
Especificidad: 0.9260103391753877
AUC: 0.7128451296599839
 
IMPORTANCIAS
--------------------------------
V2_tamp: 0.11386177742940937
V3_stmid: 0.10298733351662497
V1_qrsarea: 0.09808103501039561
V6_qtint: 0.09793609234632264
V3_qrsarea: 0.0949970402806656
I_rdur: 0.09243712875769948
aVL_qrsarea: 0.09237365641565426
V5_samp: 0.09217465108074169
V1_qrsdur: 0.0908340900971677
V5_vat: 0.07887716238988238
V5_qamp: 0.045440032675436386
 
RESULTADOS DEL CONJUNTO SELECCIONADO POR Boruta
---------------------------------------------
Acierto: 0.7347003834042108
Precision: 0.7556931245937444
Sensibilidad: 0.28048
Especificidad: 0.9553348043542219
AUC: 0.7678544147438322
 
IMPORTANCIAS
--------------------------------
Edad: 0.06328134408910627
II_qrsppk: 0.05942264564076352
V5_qrsppk: 0.0584314264758819
V3_qrsppk:

### XGBoost

In [25]:
cars_aic = ['V5_qamp', 'I_rdur', 'V5_samp', 'V5_vat', 'V1_qrsdur', 'aVL_qrsarea', 'V1_qrsarea', 'V3_qrsarea', 'V3_stmid', 'V2_tamp', 'V6_qtint']
cars_bor = ['I_ramp', 'V5_samp', 'I_qrsppk', 'II_qrsppk', 'aVR_qrsppk', 'V1_qrsppk',
'V3_qrsppk', 'V4_qrsppk', 'V5_qrsppk', 'V1_qrsdur', 'V5_qrsarea', 'V4_stmid',
'V2_tamp', 'V3_tamp', 'aVL_tdur', 'III_qtint', 'V6_qtint', 'Edad'
]

caracteristicas = [cars_aic, cars_bor]
models = ['AIC', 'Boruta']

params_aic = {'n_estimators': 250, 'max_leaves': 0, 'max_depth': 25, 'learning_rate': 0.1}
params_bor = {'n_estimators': 250, 'max_leaves': 0, 'max_depth': 50, 'learning_rate': 0.25}
params = [params_aic, params_bor]

modelos = []
val_test = []

for i in range(len(caracteristicas)):
    
    x2 = x[caracteristicas[i]]
    
    aucs = []
    exs = []
    precs = []
    esps = []
    sens = []

    for j in range(100):
        
        kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=j)
        
        for fold, (train, test) in enumerate(kf.split(x2, y)):
            
            x_t, x_ts = x2.values[train], x2.values[test]
            y_t, y_ts = y[train], y[test]
            
            y_t = y_t.reset_index(drop=True)
            y_ts = y_ts.reset_index(drop=True)
            
            pesos = (len(y_t[y_t == 0]) / len(y_t[y_t==1]))

            modelo = xgb.XGBClassifier(device="cuda", n_jobs=-1, eval_metric=roc_auc_score, scale_pos_weight=pesos, 
                              n_estimators=params[i]['n_estimators'], max_leaves=params[i]['max_leaves'], 
                                       max_depth=params[i]['max_depth'], learning_rate=params[i]['learning_rate'])
            modelo.fit(cp.array(x_t), y_t)
            
            prob = modelo.predict_proba(cp.array(x_ts))
            pred = modelo.predict(cp.array(x_ts))
                      
            tn, fp, fn, tp = confusion_matrix(y_ts, pred).ravel()
            auc = roc_auc_score(y_ts, prob[:,1])
            prec = precision_score(y_ts, pred, zero_division=0)
            ex = accuracy_score(y_ts, pred)
            sen = recall_score(y_ts, pred)
            esp = tn / (tn + fp)
            
            aucs.append(auc)
            exs.append(ex)
            precs.append(prec)
            sens.append(sen)
            esps.append(esp)        
    
    print(f' ')
    print(f'RESULTADOS DEL CONJUNTO SELECCIONADO POR {models[i]}')
    print(f'---------------------------------------------')
    print(f'Acierto: {np.mean(exs)}')
    print(f'Precision: {np.mean(precs)}')
    print(f'Sensibilidad: {np.mean(sens)}')
    print(f'Especificidad: {np.mean(esps)}')
    print(f'AUC: {np.mean(aucs)}')
    
    x_train_2 = x_train[caracteristicas[i]]
    
    modelo = xgb.XGBClassifier(device="cuda", n_jobs=-1, eval_metric=roc_auc_score, scale_pos_weight=pesos, 
                              n_estimators=params[i]['n_estimators'], max_leaves=params[i]['max_leaves'], 
                                       max_depth=params[i]['max_depth'], learning_rate=params[i]['learning_rate'])
    modelo.fit(cp.array(x_train_2), y_train)
    
    modelos.append(modelo)
    val_test.append(x_test[caracteristicas[i]])
    
    cars_x = x_train_2.columns
    importancias = modelo.feature_importances_

    car_imp = sorted(zip(cars_x, importancias), key=lambda x: x[1], reverse=True)

    print(f' ')
    print(f'IMPORTANCIAS')
    print(f'--------------------------------')
    for car, importancia in car_imp:
        print(f'{car}: {importancia}')
    

 
RESULTADOS DEL CONJUNTO SELECCIONADO POR AIC
---------------------------------------------
Acierto: 0.6987547830388827
Precision: 0.5527803829919612
Sensibilidad: 0.41679999999999995
Especificidad: 0.8357144958601268
AUC: 0.7075518397567911
 
IMPORTANCIAS
--------------------------------
V5_qamp: 0.09789907187223434
V2_tamp: 0.09751595556735992
V5_vat: 0.09741503745317459
V1_qrsdur: 0.09694358706474304
V5_samp: 0.09652077406644821
V1_qrsarea: 0.08896959573030472
V3_stmid: 0.08675897121429443
V6_qtint: 0.08582401275634766
I_rdur: 0.08518597483634949
V3_qrsarea: 0.08486177027225494
aVL_qrsarea: 0.08210519701242447
 
RESULTADOS DEL CONJUNTO SELECCIONADO POR Boruta
---------------------------------------------
Acierto: 0.7384473937076043
Precision: 0.6312983686702999
Sensibilidad: 0.48282666666666674
Especificidad: 0.8626142983230362
AUC: 0.7616004578377394
 
IMPORTANCIAS
--------------------------------
V1_qrsdur: 0.06616365164518356
V5_qrsarea: 0.0638175830245018
II_qrsppk: 0.061828657

---

# Regresión

### Regresión lineal

In [112]:
cars_aic = ['I_rdur', 'I_qrsarea', 'V1_qrsarea', 'V2_stdur', 'V2_tamp', 'V6_qtint']
cars_bor = ['V1_pamp', 'II_ramp', 'V5_samp', 'III_vat', 'V1_vat', 'I_qrsppk', 'aVF_qrsppk', 'V1_qrsppk', 'V4_qrsppk', 'V2_qrsarea', 'I_stmid', 'V3_stmid', 'aVR_tamp', 'V1_tamp', 'V2_tamp', 'V3_tamp', 'I_tdur', 'aVL_tdur', 'V6_tdur', 'Edad']

caracteristicas = [cars_aic, cars_bor]
models = ['AIC', 'Boruta']

for i in range(len(caracteristicas)):
    
    x2 = x[caracteristicas[i]]
    x2 = sm.add_constant(x2)
    
    r2s = []
    mses = []
    rmses = []

    for j in range(100):
        
        kf = KFold(n_splits=5 shuffle=True, random_state=j)
        
        for fold, (train, test) in enumerate(kf.split(x2, y)):
            
            x_t, x_ts = x2.values[train], x2.values[test]
            y_t, y_ts = y[train], y[test]
            
            y_t = y_t.reset_index(drop=True)
            y_ts = y_ts.reset_index(drop=True)
        
            modelo = sm.OLS(y_t, x_t)
            modelo = modelo.fit()
            
            pred = modelo.predict(x_ts)
            r2 = modelo.rsquared
            mse = mean_squared_error(y_ts, pred)
            rmse = np.sqrt(mse)
            
            r2s.append(r2)
            mses.append(mse)
            rmses.append(rmse)
    
    print(f' ')
    print(f'RESULTADOS DEL CONJUNTO SELECCIONADO POR {models[i]}')
    print(f'---------------------------------------------')
    print(f'R2: {np.mean(r2s)}')
    print(f'MSE: {np.mean(mses)}')
    print(f'RMSE: {np.mean(rmses)}')
    
    x_train_2 = x_train[caracteristicas[i]]
    x_train_2 = sm.add_constant(x_train_2)
    
    y_train = y_train.reset_index(drop=True)

    modelo = sm.OLS(y_train, x_train_2)
    modelo = modelo.fit()
    print(modelo.summary())

 
RESULTADOS DEL CONJUNTO SELECCIONADO POR AIC
---------------------------------------------
R2: 0.037137737724426564
MSE: 0.6282607373609659
RMSE: 0.7924485529645218
                            OLS Regression Results                            
Dep. Variable:                      K   R-squared:                       0.046
Model:                            OLS   Adj. R-squared:                  0.043
Method:                 Least Squares   F-statistic:                     15.84
Date:                Sun, 30 Jun 2024   Prob (F-statistic):           7.47e-18
Time:                        12:53:57   Log-Likelihood:                -2341.2
No. Observations:                1986   AIC:                             4696.
Df Residuals:                    1979   BIC:                             4736.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t

In [48]:
cars_bor = ['V1_pamp', 'II_ramp', 'V5_samp', 'III_vat', 'V1_vat', 'I_qrsppk', 'aVF_qrsppk', 'V1_qrsppk', 'V4_qrsppk', 'V2_qrsarea', 'I_stmid', 'V3_stmid', 'aVR_tamp', 'V1_tamp', 'V2_tamp', 'V3_tamp', 'I_tdur', 'aVL_tdur', 'V6_tdur', 'Edad']
x2 = x[cars_bor]

modelo = sm.OLS(y, x2)
modelo = modelo.fit()


pred = modelo.predict(x_ts)
r2 = modelo.rsquared
mse = mean_squared_error(y_ts, pred)
rmse = np.sqrt(mse)

In [55]:
pred = [np.mean(y) for i in range(len(y))]
mse = mean_squared_error(y, pred)
rmse = np.sqrt(mse)
rmse

0.804283166470187

### Árbol de Regresión

In [113]:
cars_aic = ['I_rdur', 'I_qrsarea', 'V1_qrsarea', 'V2_stdur', 'V2_tamp', 'V6_qtint']
cars_bor = ['V1_pamp', 'II_ramp', 'V5_samp', 'III_vat', 'V1_vat', 'I_qrsppk', 'aVF_qrsppk', 'V1_qrsppk', 'V4_qrsppk', 'V2_qrsarea', 'I_stmid', 'V3_stmid', 'aVR_tamp', 'V1_tamp', 'V2_tamp', 'V3_tamp', 'I_tdur', 'aVL_tdur', 'V6_tdur', 'Edad']

caracteristicas = [cars_aic, cars_bor]
models = ['AIC', 'Boruta']

params_aic = {'max_depth': 100, 'max_features': 1, 'max_leaf_nodes': 5}
params_bor = {'max_depth': 100, 'max_features': 10, 'max_leaf_nodes': 5}
params = [params_aic, params_bor]

for i in range(len(caracteristicas)):
    
    x2 = x[caracteristicas[i]]
    
    r2s = []
    mses = []
    rmses = []
    
    for j in range(100):
        
        kf = KFold(n_splits=5, shuffle=True, random_state=j)
        
        for fold, (train, test) in enumerate(kf.split(x2, y)):
            
            x_t, x_ts = x2.values[train], x2.values[test]
            y_t, y_ts = y[train], y[test]
            
            y_t = y_t.reset_index(drop=True)
            y_ts = y_ts.reset_index(drop=True)

            modelo = DecisionTreeRegressor(max_leaf_nodes=params[i]['max_leaf_nodes'], 
                                   max_features=params[i]['max_features'], max_depth=params[i]['max_depth'])
            modelo.fit(x_t, y_t)
                   
            pred = modelo.predict(x_ts)
            r2 = modelo.score(x_ts, y_ts)
            mse = mean_squared_error(y_ts, pred)
            rmse = np.sqrt(mse)
            
            r2s.append(r2)
            mses.append(mse)
            rmses.append(rmse)     
    
    print(f' ')
    print(f'RESULTADOS DEL CONJUNTO SELECCIONADO POR {models[i]}')
    print(f'---------------------------------------------')
    print(f'R2: {np.mean(r2s)}')
    print(f'MSE: {np.mean(mses)}')
    print(f'RMSE: {np.mean(rmses)}')
    
    x_train_2 = x_train[caracteristicas[i]]
    
    modelo = DecisionTreeRegressor(max_leaf_nodes=params[i]['max_leaf_nodes'], 
                                   max_features=params[i]['max_features'], max_depth=params[i]['max_depth'])
    modelo.fit(x_train_2, y_train)
    
    cars_x = x_train_2.columns
    importancias = modelo.feature_importances_

    car_imp = sorted(zip(cars_x, importancias), key=lambda x: x[1], reverse=True)

    print(f' ')
    print(f'IMPORTANCIAS')
    print(f'--------------------------------')
    for car, importancia in car_imp:
        print(f'{car}: {importancia}')
    

 
RESULTADOS DEL CONJUNTO SELECCIONADO POR AIC
---------------------------------------------
R2: -0.004420221997837593
MSE: 0.649215883231584
RMSE: 0.805556430602847
 
IMPORTANCIAS
--------------------------------
V6_qtint: 0.7642905303678522
I_rdur: 0.2357094696321478
I_qrsarea: 0.0
V1_qrsarea: 0.0
V2_stdur: 0.0
V2_tamp: 0.0
 
RESULTADOS DEL CONJUNTO SELECCIONADO POR Boruta
---------------------------------------------
R2: 0.010848507921189316
MSE: 0.639289280856932
RMSE: 0.7993694688383381
 
IMPORTANCIAS
--------------------------------
I_qrsppk: 0.29891958069926966
V3_tamp: 0.26491433172681805
III_vat: 0.25425117606132464
I_tdur: 0.1819149115125876
V1_pamp: 0.0
II_ramp: 0.0
V5_samp: 0.0
V1_vat: 0.0
aVF_qrsppk: 0.0
V1_qrsppk: 0.0
V4_qrsppk: 0.0
V2_qrsarea: 0.0
I_stmid: 0.0
V3_stmid: 0.0
aVR_tamp: 0.0
V1_tamp: 0.0
V2_tamp: 0.0
aVL_tdur: 0.0
V6_tdur: 0.0
Edad: 0.0


### Random Forest

In [114]:
cars_aic = ['I_rdur', 'I_qrsarea', 'V1_qrsarea', 'V2_stdur', 'V2_tamp', 'V6_qtint']
cars_bor = ['V1_pamp', 'II_ramp', 'V5_samp', 'III_vat', 'V1_vat', 'I_qrsppk', 'aVF_qrsppk', 'V1_qrsppk', 'V4_qrsppk', 'V2_qrsarea', 'I_stmid', 'V3_stmid', 'aVR_tamp', 'V1_tamp', 'V2_tamp', 'V3_tamp', 'I_tdur', 'aVL_tdur', 'V6_tdur', 'Edad']

caracteristicas = [cars_aic, cars_bor]
models = ['AIC', 'Boruta']

params_aic = {'n_estimators': 250, 'max_leaf_nodes': None, 'max_features': 5, 'max_depth': None}
params_bor = {'n_estimators': 200, 'max_leaf_nodes': None, 'max_features': 10, 'max_depth': None}
params = [params_aic, params_bor]

for i in range(len(caracteristicas)):
    
    x2 = x[caracteristicas[i]]
    
    r2s = []
    mses = []
    rmses = []
    
    for j in range(100):
        
        kf = KFold(n_splits=5, shuffle=True, random_state=j)
        
        for fold, (train, test) in enumerate(kf.split(x2, y)):
            
            x_t, x_ts = x2.values[train], x2.values[test]
            y_t, y_ts = y[train], y[test]
            
            y_t = y_t.reset_index(drop=True)
            y_ts = y_ts.reset_index(drop=True)

            modelo = RandomForestRegressor(n_jobs=-1, oob_score=r2_score,
                                   n_estimators=params[i]['n_estimators'], 
                                   max_leaf_nodes=params[i]['max_leaf_nodes'], max_features=params[i]['max_features'], 
                                            max_depth=params[i]['max_depth'])
            modelo.fit(x_t, y_t)
                   
            pred = modelo.predict(x_ts)
            r2 = modelo.score(x_ts, y_ts)
            mse = mean_squared_error(y_ts, pred)
            rmse = np.sqrt(mse)
            
            r2s.append(r2)
            mses.append(mse)
            rmses.append(rmse)     
    
    print(f' ')
    print(f'RESULTADOS DEL CONJUNTO SELECCIONADO POR {models[i]}')
    print(f'---------------------------------------------')
    print(f'R2: {np.mean(r2s)}%')
    print(f'MSE: {np.mean(mses)}')
    print(f'RMSE: {np.mean(rmses)}')
    
    x_train_2 = x_train[caracteristicas[i]]
    
    modelo = RandomForestRegressor(n_jobs=-1, oob_score=r2_score,
                                   n_estimators=params[i]['n_estimators'], 
                                   max_leaf_nodes=params[i]['max_leaf_nodes'], max_features=params[i]['max_features'], 
                                            max_depth=params[i]['max_depth'])
    modelo.fit(x_train_2, y_train)
    
    cars_x = x_train_2.columns
    importancias = modelo.feature_importances_

    car_imp = sorted(zip(cars_x, importancias), key=lambda x: x[1], reverse=True)

    print(f' ')
    print(f'IMPORTANCIAS')
    print(f'--------------------------------')
    for car, importancia in car_imp:
        print(f'{car}: {importancia}')
    

 
RESULTADOS DEL CONJUNTO SELECCIONADO POR AIC
---------------------------------------------
R2: 0.04694815528676868%
MSE: 0.6159962156754122
RMSE: 0.7846615773635393
 
IMPORTANCIAS
--------------------------------
V1_qrsarea: 0.18557317797745915
V2_tamp: 0.17675043481014788
V6_qtint: 0.17135416030682907
I_qrsarea: 0.16356489207358288
I_rdur: 0.15333117280187472
V2_stdur: 0.14942616203010642
 
RESULTADOS DEL CONJUNTO SELECCIONADO POR Boruta
---------------------------------------------
R2: 0.1698581481239295%
MSE: 0.5365523510336015
RMSE: 0.7323167058175282
 
IMPORTANCIAS
--------------------------------
V1_qrsppk: 0.06474588093986638
I_tdur: 0.05731962512330855
V4_qrsppk: 0.05726148242650791
I_qrsppk: 0.05447767435853719
V6_tdur: 0.052167370952914904
Edad: 0.0521244973885298
aVL_tdur: 0.05151360418561094
aVF_qrsppk: 0.05063889534333868
aVR_tamp: 0.050251692994472866
I_stmid: 0.04975611702418402
V2_tamp: 0.048871603880303
V3_tamp: 0.048508937440608295
V3_stmid: 0.048500064513967434
V2_

### XGBoost

In [117]:
cars_aic = ['I_rdur', 'I_qrsarea', 'V1_qrsarea', 'V2_stdur', 'V2_tamp', 'V6_qtint']
cars_bor = ['V1_pamp', 'II_ramp', 'V5_samp', 'III_vat', 'V1_vat', 'I_qrsppk', 'aVF_qrsppk', 'V1_qrsppk', 'V4_qrsppk', 'V2_qrsarea', 'I_stmid', 'V3_stmid', 'aVR_tamp', 'V1_tamp', 'V2_tamp', 'V3_tamp', 'I_tdur', 'aVL_tdur', 'V6_tdur', 'Edad']

caracteristicas = [cars_aic, cars_bor]
models = ['AIC', 'Boruta']

params_aic = {'n_estimators': 150, 'max_leaves': 10, 'max_depth': 25, 'learning_rate': 0.1}
params_bor = {'n_estimators': 200, 'max_leaves': 0, 'max_depth': 10, 'learning_rate': 0.1}
params = [params_aic, params_bor]

for i in range(len(caracteristicas)):
    
    x2 = x[caracteristicas[i]]
    
    r2s = []
    mses = []
    rmses = []
    
    for j in range(100):
        
        kf = KFold(n_splits=5, shuffle=True, random_state=j)
        
        for fold, (train, test) in enumerate(kf.split(x2, y)):
            
            x_t, x_ts = x2.values[train], x2.values[test]
            y_t, y_ts = y[train], y[test]
            
            y_t = y_t.reset_index(drop=True)
            y_ts = y_ts.reset_index(drop=True)

            modelo = xgb.XGBRegressor(device="cuda", n_jobs=-1, eval_metric=r2_score,
                              n_estimators=params[i]['n_estimators'], max_leaves=params[i]['max_leaves'], 
                                       max_depth=params[i]['max_depth'], learning_rate=params[i]['learning_rate'])
            modelo.fit(cp.array(x_t), y_t)
                   
            pred = modelo.predict(cp.array(x_ts))
            r2 = modelo.score(cp.array(x_ts), y_ts)
            mse = mean_squared_error(y_ts, pred)
            rmse = np.sqrt(mse)
            
            r2s.append(r2)
            mses.append(mse)
            rmses.append(rmse)     
    
    print(f' ')
    print(f'RESULTADOS DEL CONJUNTO SELECCIONADO POR {models[i]}')
    print(f'---------------------------------------------')
    print(f'R2: {np.mean(r2s)}')
    print(f'MSE: {np.mean(mses)}')
    print(f'RMSE: {np.mean(rmses)}')
    
    x_train_2 = x_train[caracteristicas[i]]
    
    modelo = xgb.XGBRegressor(device="cuda", n_jobs=-1, eval_metric=r2_score,
                              n_estimators=params[i]['n_estimators'], max_leaves=params[i]['max_leaves'], 
                                       max_depth=params[i]['max_depth'], learning_rate=params[i]['learning_rate'])
    modelo.fit(cp.array(x_train_2), y_train)
    
    cars_x = x_train_2.columns
    importancias = modelo.feature_importances_

    car_imp = sorted(zip(cars_x, importancias), key=lambda x: x[1], reverse=True)

    print(f' ')
    print(f'IMPORTANCIAS')
    print(f'--------------------------------')
    for car, importancia in car_imp:
        print(f'{car}: {importancia}')
    

 
RESULTADOS DEL CONJUNTO SELECCIONADO POR AIC
---------------------------------------------
R2: 0.013304536961598702
MSE: 0.637660489711632
RMSE: 0.79834922623642
 
IMPORTANCIAS
--------------------------------
V1_qrsarea: 0.19033032655715942
V2_stdur: 0.17765352129936218
V6_qtint: 0.16998641192913055
V2_tamp: 0.1661233901977539
I_qrsarea: 0.15862028300762177
I_rdur: 0.13728618621826172
 
RESULTADOS DEL CONJUNTO SELECCIONADO POR Boruta
---------------------------------------------
R2: 0.1400127290477273
MSE: 0.5557167034564324
RMSE: 0.7452504102836868
 
IMPORTANCIAS
--------------------------------
I_tdur: 0.07049322873353958
V1_qrsppk: 0.06644800305366516
V3_tamp: 0.05982399359345436
V6_tdur: 0.05753558501601219
aVR_tamp: 0.05708673596382141
V1_tamp: 0.05706852301955223
Edad: 0.05642324686050415
V3_stmid: 0.05356409773230553
V4_qrsppk: 0.05267159640789032
V2_tamp: 0.05244767665863037
aVL_tdur: 0.05175670608878136
I_qrsppk: 0.0497768260538578
aVF_qrsppk: 0.04937208071351051
V1_vat: 0.